# Sistemas de recomendaciones
---
RIIAA 2019


Andong Luis Li Zhao


Las metas de este mini curso son:
*  Aprender cómo utilizar aprendizaje de máquina en sistemas de recomendaciones
*  Implementar e utilizar técnicas comunes en sistemas de recomendaciones
*  Usar Python y pandas para manipular datos e implementar algoritmos
*  Explorar el panorama general de problemas en sistemas de recomendaciones 

Nota: Usaremos pandas bastante en este tutorial. Si no estas familiarizado con esta biblioteca, recomiendo hacer un pequeño repaso de manipulación (y usar Google bastante) https://pandas.pydata.org/pandas-docs/stable/

## Repaso: Aprendizaje de máquina




In [0]:
# Obtener base de datos
import pandas as pd

iris = pd.read_csv('https://raw.githubusercontent.com/mwaskom/seaborn-data/master/iris.csv')

print(iris.shape)
print(iris.head())

In [0]:
# Creación de datos para entrenamiento y para evaluación
from sklearn.model_selection import train_test_split

y = iris["species"]
X = iris.drop("species", axis=1)

trainX, testX, trainY, testY = train_test_split(X, y, test_size=0.25)

print(trainX.shape)
print(testX.shape)
print(trainY.shape)
print(testY.shape)


In [0]:
# Creación y entrenamiento de modelo

from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier()

clf.fit(trainX, trainY)


In [0]:
# Evaluación de modelo
from sklearn import metrics

train_pred = clf.predict(trainX)
test_pred = clf.predict(testX)
print(f"Train accuracy score: {metrics.accuracy_score(trainY, train_pred)}")
print(f"Test accuracy score: {metrics.accuracy_score(testY, test_pred)}")

print("Train Confusion Matrix")
print(metrics.confusion_matrix(trainY, train_pred))
print("Test Confusion Matrix")
print(metrics.confusion_matrix(testY, test_pred))

In [0]:
# Validación cruzada

from sklearn.model_selection import cross_val_score

cv_results = cross_val_score(clf, trainX, trainY, cv=3, scoring="accuracy")
print(cv_results)

## Repaso: Álgebra lineal

In [0]:
# Matrices
import numpy as np

A = np.random.randint(0, 10, size=(3,5))
print(A)
print(A.shape)

In [0]:
# Accesar elementos de matrices en numpy
print(A[0])
print(A[0,1])
print(A[:, 1])
print(A[0, :])

In [0]:
# Matriz transpuesta
print(A.T)
print(A.T.shape)

In [0]:
B = np.random.randint(0, 10, size=(5,5))
print(B)
print(A.shape)
print(B.shape)

In [0]:
# Producto de Matrices
print(A @ B)

In [0]:
# Producto de Matrices
print(B @ A)

In [0]:
# Producto de Matrices
print(B @ A.T)

In [0]:
# Rango

print(np.linalg.matrix_rank(A))
print(np.linalg.matrix_rank(A.T))
print(np.linalg.matrix_rank(B))
print(np.linalg.matrix_rank(B.T))

## Procesamiento de datos

In [0]:
# Obtener la base de datos (subir archivos desde tu computadora)
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  

In [0]:
# Leer los datos
from io import StringIO
import pandas as pd

# Ratings/valoraciones
s=str(uploaded["ratings.csv"],'utf-8')
data = StringIO(s) 
r_df = pd.read_csv(data)
r_df = r_df.sample(frac=1, random_state=43)

# Géneros
s=str(uploaded["genres.csv"],'utf-8')
data = StringIO(s) 
g_df = pd.read_csv(data)





In [0]:
# Imprimir algunos datos

print(r_df.shape)
print(list(r_df))
print(r_df.head())

print(g_df.shape)
print(list(g_df))
print(g_df.head())

In [0]:
# Separar en datos de entrenamiento y validación
split_ind = 9 * r_df.shape[0] // 10

test_df = r_df.iloc[split_ind:, :]
r_df = r_df.iloc[0:split_ind, :]

print(test_df.shape)
print(r_df.shape)

## k Nearest Neighbors


In [0]:
# Obtener datos que se usaran
r_df = r_df # No hace nada esta linea, nada mas para señalar qué datos usamos
test_df = test_df

In [0]:
# Hacer matriz de usuarios y películas

ui_matrix = r_df.pivot(index='userId', columns='movieId', values='rating')

print(test_df.shape)
print(r_df.shape)
print(ui_matrix.shape)

In [0]:
import numpy as np
import pandas as pd
from scipy.spatial.distance import cosine


def jaccard_simil(ratings_1, ratings_2):
    # Input: two 1D arrays, same length. No NaNs, 
    #        but 0's indicate no rating
    # Output: Jaccard similarity
    
    # TODO: Calcula la similitud de jaccard
    return -1


def cosine_simil(ratings_1, ratings_2):
    # Input: two 1D arrays, same length. With NaN's
    # Output: Jaccard similarity
    
    # TODO: Calcula la similitud de coseno con función cosine() importada
    return -1

# Clase principal para predicción con kNN
class KnnClass():


    def __init__(self, min_val, max_val, k=5, similarity="pearson"):

        simils = ["pearson", "cosine", "jaccard"]
        if similarity not in simils:
            raise ValueError('Error: similitud invalida, debe ser ["pearson", "cosine", "jaccard"]')

        self.similarity = similarity
        self.k = k
        self.means = pd.Series()
        self.ratings = pd.DataFrame()
        self.similarities = pd.DataFrame()
        self.min_val = min_val
        self.max_val = max_val


    def fit(self, df):
        # df es DataFrame donde cada fila es un usuario y cada columna es una película
        # Los valores de la celda indican la valoración de ese usuario sobre esa película
        # Hay valores NaN si la valoración no existe
        
        # Guardar valoraciones para las predicciones
        self.ratings = df

        # Calcular promedios
        self.means = self.ratings.mean(axis=1, skipna=True)

        # Calcular similitudes
        if self.similarity == "pearson":
            self.similarities = self.ratings.T.corr(method="pearson")

        elif self.similarity == "jaccard":
            # Aquí convertimos de NaN's a 0's porque la función jaccard_simil
            # asume eso. Esto es necesario para que .corr() calcule la similitud
            # correctamente
            self.similarities = self.ratings.T.fillna(0).corr(method=jaccard_simil)

        elif self.similarity == "cosine":
            self.similarities = self.ratings.T.corr(method=cosine_simil)


    def _prior(self, user_id, item_id):
        # Predicción si user_id y/o item_id son nuevos en el sistema
        if user_id not in self.ratings.index:
            if item_id not in list(self.ratings):
                # Si ambos son nuevos, regresa punto medio
                return (self.min_val + self.max_val) / 2
            else:
                # Si usuario es nuevo, regresa promedio de la película
                return self.ratings[item_id].mean(skipna=True)
        else:
            # Si la película es nueva, regresa promedio del usuario
            return self.ratings.loc[user_id, :].mean(skipna=True)



    def predict(self, user_id, item_id, round_pred=False):

        # Checar si usuario/película es nueva
        if user_id not in self.ratings.index or item_id not in self.ratings:
            pred =  self._prior(user_id, item_id)

        else:
            
            # Obten similitudes para usuario que han valorado item_id
            raters = self.ratings[~ self.ratings[item_id].isnull()].index
            simils = self.similarities.loc[user_id, raters]

            # Ordena similitudes en orden descendiente
            simils = simils.sort_values(ascending=False)

            # Obten los k usuarios con similitudes a user_id más grandes 
            inds = simils.nlargest(min(self.k, len(simils))).index


            # Calcular predicción
            if simils[inds].sum() != 0:
                # TODO: Calcula predicción
                pred = -1
            else:
                # TODO: Calcula predicción cuando la suma de abajo es igual a 0
                pred = -1

            # Truncate just in case
            # Asegurate que la predicción esté entre max_val y min_val
            pred = max(min(pred, self.max_val), self.min_val)

        # Devuelve la predicción (redondeada si es que es necesario)
        if round_pred:
            return round(pred, 1)
        else:
            return pred


In [0]:
# Unit tests para coeficiente Pearson (todo debería de pasar, ya estaba implementada)

# Test 1
knn = KnnClass(min_val=1.0, max_val=5.0, k=20, similarity="pearson")
knn.fit(ui_matrix)
val = knn.similarities.loc[1, 1]
assert val == 1.0, f"Test 1: val debería ser 1.0, en vez de {val}"

# Test 2
val = knn.similarities.loc[1, 4]
assert val == 0.48723001369349506, f"Test 2: val debería ser 0.48723001369349506, en vez de {val}"

# Test 3
val = knn.similarities.loc[2, 2]
assert val == 1.0, f"Test 3: val debería ser 1.0, en vez de {val}"

# Test 4
val = knn.similarities.loc[2, 550]
assert val == -0.6071428571428572, f"Test 4: val debería ser -0.6071428571428572, en vez de {val}"


In [0]:
# Unit tests para similaridades: Coseno

# Test 1
knn = KnnClass(min_val=1.0, max_val=5.0, k=20, similarity="cosine")
knn.fit(ui_matrix)
val = knn.similarities.loc[1, 1]
assert val == 1.0, f"Test 1: val debería ser 1.0, en vez de {val}"

# Test 2
val = knn.similarities.loc[1, 4]
assert val == 0.9424549558857006, f"Test 2: val debería ser 0.9424549558857006, en vez de {val}"

# Test 3
val = knn.similarities.loc[2, 2]
assert val == 1.0, f"Test 3: val debería ser 1.0, en vez de {val}"

# Test 4
val = knn.similarities.loc[2, 550]
assert val == 0.9671696756264027, f"Test 4: val debería ser 0.9671696756264027, en vez de {val}"


In [0]:
# Unit tests para similaridades: Jaccard

# Test 1
knn = KnnClass(min_val=1.0, max_val=5.0, k=20, similarity="jaccard")
knn.fit(ui_matrix)
val = knn.similarities.loc[1, 1]
assert val == 1.0, f"Test 1: val debería ser 1.0, en vez de {val}"

# Test 2
val = knn.similarities.loc[1, 4]
assert val == 0.08189655172413793, f"Test 2: val debería ser 1.0, en vez de {val}"

# Test 3
val = knn.similarities.loc[2, 2]
assert val == 1.0, f"Test 3: val debería ser 1.0, en vez de {val}"

# Test 4
val = knn.similarities.loc[2, 550]
assert val == 0.17857142857142858, f"Test 4: val debería ser 1.0, en vez de {val}"

In [0]:
# Unit test para predict
# Si tienes los siguientes resultados, entonces probablemente está correcta tu implementación

# Test 1
knn = KnnClass(min_val=1.0, max_val=5.0, k=20, similarity="pearson")
knn.fit(ui_matrix)
pred = knn.predict(user_id=1, item_id=1, round_pred=True)
assert pred == 4.7, f"Test 1: pred debería ser 4.7, en vez de {pred}"

# Test 2
pred = knn.predict(user_id=2, item_id=5, round_pred=True)
assert pred == 5.0, f"Test 2: pred debería ser 5.0, en vez de {pred}"

# Test 3
knn = KnnClass(min_val=1.0, max_val=5.0, k=5, similarity="cosine")
knn.fit(ui_matrix)
pred = knn.predict(user_id=1, item_id=1, round_pred=True)
assert pred == 4.2, f"Test 3 Error: pred debería ser 4.2, en vez de {pred}"

# Test 4
knn = KnnClass(min_val=1.0, max_val=5.0, k=10, similarity="jaccard")
knn.fit(ui_matrix)
pred = knn.predict(user_id=1, item_id=1, round_pred=True)
assert pred == 5.0, f"Test 4 Error: pred debería ser 5.0, en vez de {pred}"

# Test 5
pred = knn.predict(user_id=2, item_id=5, round_pred=True)
assert pred == 1.0, f"Test 5 Error: pred debería ser 1.0, en vez de {pred}"


In [0]:
# Hacer grafica de diferentes ks

%matplotlib inline

from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt


ks = [1, 2, 5, 10, 20, 50]
res_pearson = [0] * len(ks)
res_cosine = [0] * len(ks)
res_jaccard = [0] * len(ks)

results = [res_pearson, res_cosine, res_jaccard]

for i, k in enumerate(ks):
  for j, simil in enumerate(["pearson", "cosine", "jaccard"]):
    knn = KnnClass(min_val=1.0, max_val=5.0, k=k, similarity=simil)
    knn.fit(ui_matrix)

    predictions= test_df.apply(lambda x: knn.predict(user_id=x["userId"], item_id=x["movieId"], round_pred=True), axis=1)

    mae = mean_absolute_error(test_df["rating"], predictions)
    
    results[j][i] = mae
    
    


plt.figure()
plt.plot(ks, res_pearson, color='r', label='Pearson')
plt.plot(ks, res_cosine, color='b', label='Coseno')
plt.plot(ks, res_jaccard, color='g', label='Jaccard')
plt.xlabel("Número de vecinos (k)")
plt.ylabel("Error absoluto medio")
plt.legend()
plt.title("Rendimiento de kNN con diferentes hiperparametros")
plt.show()
plt.clf()



## Factorización de matrices: demo


In [0]:
# Ejemplo de factorización SVD

import numpy as np

A = np.array([[5, 1, 0, 3, 0],[1, 2, 2, 4, 5],[0, 3, 1, 2, 0] ])

U, s, VT = np.linalg.svd(A)

# NOTA: s es un vector de los valores singulares
S = np.zeros(A.shape)
S[:A.shape[0], :A.shape[0]] = np.diag(s)

print(f"U: {U}")
print(f"S: {S}")
print(f"VT: {VT}")

In [0]:
# Producto U @ S @ VT

print(f"A: {A}")
print(f"Producto: {np.round(U @ S @ VT)}")

In [0]:
# Ejemplo de aproximación
S[2,2] = 0

print(f"A: {A}")
print(f"Approx: {np.round(U @ S @ VT)}")

## Factorización de matrices

In [0]:
# Obtener datos que se usaran 

# No hacen nada estas lineas, nada más son para señalar qué datos usamos
r_df = r_df
ui_matrix = ui_matrix
test_df = test_df


print(test_df.shape)
print(r_df.shape)
print(ui_matrix.shape)

In [0]:
# Clase principal para el modelo SVD


class SVDClass():

    def __init__(self, min_val, max_val, r=100, ):

        self.r = r
        self.ratings = pd.DataFrame()
        self.predictions = pd.DataFrame()
        self.min_val = min_val
        self.max_val = max_val


    def fit(self, df):

        # Guardar valoraciones para _prior()
        self.ratings = df
        
        # Crea matriz con valores de df
        Y = df.values

        # TODO: Reemplaza NaN's con 0's

        # Calcular U, s, VT con np.linalg.svd
        # NOTA: s es un vector de valores singulares
        # s ya esta en orden descendiente
        U, s, VT = np.linalg.svd(Y)

        # TODO: Conserva solo los r valores singulares más grandes de s

        # TODO: Crea la matriz diagonal S con dimensiones correctas

        # TODO: Calcula la aproximación de y

        # Crear un DataFrame en base a la aproximación
        self.predictions = pd.DataFrame(data=Y_hat,
                                        index=df.index,
                                        columns=list(df))

    def _prior(self, user_id, item_id):
        # Predicción si user_id y/o item_id son nuevos en el sistema
        if user_id not in self.ratings.index:
            if item_id not in list(self.ratings):
                return (self.min_val + self.max_val) / 2
            else:
                return self.ratings[item_id].mean(skipna=True)
        else:
            return self.ratings.loc[user_id, :].mean(skipna=True)



    def predict(self, user_id, item_id, round_pred=True):

        # Checar si es un nuevo usuario o nueva película
        if user_id not in self.ratings.index or item_id not in self.ratings:
            pred =  self._prior(user_id, item_id)
        else:
            # TODO: Predice la valoración de user_id sobre item_id
            pred = -1

            # TODO: Asegurate que la predicción este entre min_val y max_val
            pred = pred          

        # Redonde la predicción y devuelvela
        if round_pred:
            return round(pred, 1)
        else:
            return pred

In [0]:
# Unit test para cada funcion

svd_model = SVDClass(min_val=1.0, max_val=5.0, r=1,)
svd_model.fit(ui_matrix)
# Test 1
pred = svd_model.predict(user_id=1, item_id=1, round_pred=True)
assert pred == 1.3, f"Test 1 Error: pred debería ser 1.3, en vez de {pred}"

# Test 2
pred = svd_model.predict(user_id=2, item_id=5, round_pred=True)
assert pred == 1.0, f"Test 2 Error: pred debería ser 1.0, en vez de {pred}"


svd_model = SVDClass(min_val=1.0, max_val=5.0, r=10,)
svd_model.fit(ui_matrix)

# Test 3
pred = svd_model.predict(user_id=1, item_id=1, round_pred=True)
assert pred == 2.2, f"Test 3 Error: pred debería ser 2.2, en vez de {pred}"

# Test 4
pred = svd_model.predict(user_id=2, item_id=5, round_pred=True)
assert pred == 1.0, f"Test 4 Error: pred debería ser 1.0, en vez de {pred}"


svd_model = SVDClass(min_val=1.0, max_val=5.0, r=40,)
svd_model.fit(ui_matrix)

# Test 5
pred = svd_model.predict(user_id=1, item_id=1, round_pred=True)
assert pred == 2.8, f"Test 5 Error: pred debería ser 2.8, en vez de {pred}"

# Test 6
pred = svd_model.predict(user_id=2, item_id=5, round_pred=True)
assert pred == 1.0, f"Test 6 Error: pred debería ser 1.0, en vez de {pred}"

In [0]:
# Hacer grafica de diferentes ks

rs = [1, 2, 5, 10, 20, 50, 100, 200, 300, 600]
results = [0] * len(rs)

for i, r in enumerate(rs):
    svd_model = SVDClass(min_val=1.0, max_val=5.0, r=r,)
    svd_model.fit(ui_matrix)

    predictions= test_df.apply(lambda x: svd_model.predict(user_id=x["userId"], item_id=x["movieId"], round_pred=True), axis=1)

    mae = mean_absolute_error(test_df["rating"], predictions)
    
    results[i] = mae
    
plt.figure()
plt.plot(rs, results, color='r', label='SVD')
plt.xlabel("Número de valores singulares")
plt.ylabel("Error absoluto medio")
plt.legend()
plt.title("Rendimiento de SVD con diferente número de valores singulares")
plt.show()


## Ingeniería de características

In [0]:
# Obtener datos que se usaran y poner en formato adecuado

# No hacen nada estas lineas, nada más son para señalar qué datos usamos
r_df = r_df
g_df = g_df

In [0]:
# Creacion de caracteristicas
from sklearn.preprocessing import MultiLabelBinarizer

class MLClass():


	def __init__(self, min_val, max_val, model):
		# NOTA: model ya debe de estar construido

		self.min_val = min_val
		self.max_val = max_val

		self.genres_df = pd.DataFrame()

		self.user_means = pd.DataFrame()

		self.movie_means = pd.DataFrame()

		self.model = model



	def _make_train_data(self, df, genres_df):
    # df será lista de valoraciones, basicamente leido directo de csv (a comparación de las otras dos clases)
		df = df.drop("timestamp", axis=1)

		# Processar datos de género de película
		genres_df = genres_df.drop("title", axis=1)
		genres_df["genres"] = genres_df["genres"].apply(lambda x: x.split("|"))
		mlb = MultiLabelBinarizer()
		genre_labels = pd.DataFrame(mlb.fit_transform(genres_df["genres"]),columns=mlb.classes_, index=genres_df.index)
		genres_df = genres_df.drop("genres", axis=1)
		genres_df = genres_df.merge(genre_labels, left_index=True, right_index=True)

		self.genres_df = genres_df

		# Obtener promedio de usuario y películas
		self.user_means = df.groupby("userId")[["rating"]].mean().rename(columns={"rating":"user_mean"})
		self.movie_means = df.groupby("movieId")[["rating"]].mean().rename(columns={"rating":"movie_mean"}) 
    
    # Adjuntar información de usuario y películas en vectores de características "tradicionales"
		X = df.merge(genres_df, left_on="movieId", right_on="movieId")
		X = X.merge(self.user_means, left_on="userId", right_on="userId")
		X = X.merge(self.movie_means, left_on="movieId", right_on="movieId")

		y = X["rating"]
		X = X.drop("rating", axis=1)

		return X, y



	def fit(self, df, genres_df):
		# Hacer vectores de características
		X, y = self._make_train_data(df, genres_df)

		# Entrenar modelo
		self.model.fit(X, y)


	def predict(self, user_id, item_id, round_pred=False):
    
    # Crear vectores de características
		x = pd.DataFrame({"userId":[user_id], "movieId":[item_id]})
		x = x.merge(self.genres_df, left_on="movieId", right_on="movieId")

		if user_id not in self.user_means.index:
			x["user_mean"] = (self.min_val + self.max_val) / 2
			x["no_user_ratings"] = 0
		else:
			x = x.merge(self.user_means, left_on="userId", right_index=True,)

		if item_id not in self.movie_means.index:
			x["movie_mean"] = (self.min_val + self.max_val) / 2
			x["no_movie_ratings"] =0.0
		else:
			x = x.merge(self.movie_means, left_on="movieId", right_index=True,)		

		# Obtener predicción del modelo
		pred = self.model.predict(x)[0]

		# Asegurar que la predicción esté en el rango adecuado
		pred = max(min(pred, self.max_val), self.min_val)			

    # Redondear y devolver
		if round_pred:
			return round(pred, 1)
		else:
			return pred

In [0]:
# Unit test de caracteristicas
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

import random
random.seed(43)
np.random.seed(43)

model = LinearRegression()
model = MLClass(min_val=1.0, max_val=5.0, model=model)
model.fit(r_df, g_df)

# Test 1
pred = model.predict(user_id=1, item_id=1, round_pred=True)
assert pred == 4.6, f"Test 3 Error: pred debería ser 4.6, en vez de {pred}"

# Test 2
pred = model.predict(user_id=2, item_id=5, round_pred=True)
assert pred == 3.5, f"Test 4 Error: pred debería ser 3.5, en vez de {pred}"


rf = RandomForestRegressor()
rf_model = MLClass(min_val=1.0, max_val=5.0, model=rf)
rf_model.fit(r_df, g_df)

# Test 3
pred = rf_model.predict(user_id=1, item_id=1, round_pred=True)
assert pred == 4.4, f"Test 3 Error: pred debería ser 4.4, en vez de {pred}"

# Test 4
pred = rf_model.predict(user_id=2, item_id=5, round_pred=True)
assert pred == 2.8, f"Test 4 Error: pred debería ser 2.8, en vez de {pred}"

In [0]:
# Prediccion con bosques aleatorios

predictions= test_df.apply(lambda x: rf_model.predict(user_id=x["userId"], item_id=x["movieId"], round_pred=True), axis=1)
mae = mean_absolute_error(test_df["rating"], predictions)
print("Test MAE: {}".format(mae))

In [0]:
# Prediccion con regresión lineal

predictions= test_df.apply(lambda x: model.predict(user_id=x["userId"], item_id=x["movieId"], round_pred=True), axis=1)
mae = mean_absolute_error(test_df["rating"], predictions)
print("Test MAE: {}".format(mae))

## Surprise

In [0]:
# Instalar
!pip install scikit-surprise

In [0]:
# Obtener datos que se usaran y poner en formato adecuado

from surprise import Dataset
from surprise import Reader


# Un reader es un objeto para leer una clase
reader = Reader(rating_scale=(1, 5))

# Los Datasets son objetos que se pueden usar para hacer validación cruzada
# Son necesario para poder utilizar los modelos de la librería
train_data = Dataset.load_from_df(r_df[['userId', 'movieId', 'rating']], reader)
test_data = Dataset.load_from_df(test_df[['userId', 'movieId', 'rating']], reader)

# Además de usar Datasets, se requieren usar trainset para entrenar y testset para predecir
trainset = train_data.build_full_trainset()
testset = test_data.build_full_trainset().build_testset()


In [0]:
# Demo con kNN
from surprise import KNNWithZScore
from surprise import accuracy

# Creación del modelo
model = KNNWithZScore(k=20)

# Entrenar modelo con trainset
model.fit(trainset)

# Crear predicciones
# NOTA: preds es una lista de tuples, i.e. [(userId, movieId, real rating, estimated rating, {detalles}), ...]
preds = model.test(testset)

print(preds[0][3])

# Con los métodos de surprise.accuracy se puede medir el rendimiento del modelo
accuracy.mae(preds)

# Si quieres crear una lista de las predicciones, tienes que accesarlo así
pred_ratings = [pred[3] for pred in preds]
print(pred_ratings)

In [0]:
# Demo con SVD
# Demo con kNN
from surprise import SVD
from surprise import accuracy

# Creación del modelo
model = SVD()

# Entrenar modelo con trainset
model.fit(trainset)

# Crear predicciones
# NOTA: preds es una lista de tuples, i.e. [(userId, movieId, real rating, estimated rating, {detalles}), ...]
preds = model.test(testset)

print(preds[0][3])

# Con los métodos de surprise.accuracy se puede medir el rendimiento del modelo
accuracy.mae(preds)

# Si quieres crear una lista de las predicciones, tienes que accesarlo así
pred_ratings = [pred[3] for pred in preds]
print(pred_ratings)

# Si quieres crear un df de predicciones, se puede hacer asi
pred_df = pd.DataFrame(pred_ratings)
print(pred_df.head())

In [0]:
# Validación cruzada
from surprise.model_selection import cross_validate

cross_validate(model, train_data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

## Reto de programación

In [0]:
# Obtener la base de datos (uploading)
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  

In [0]:
# Datos

from io import StringIO
import pandas as pd

s=str(uploaded["heldout.csv"],'utf-8')
data = StringIO(s) 

df = pd.read_csv(data)

print(df)



In [0]:
'''
TODO: Crea y entrena un modelo para hacer predicciones sobre heldout.csv
Cuando esten listos, suban sus predicciones al Google Drive

La persona con mejor puntuación (Error absoluto medio más bajo) ganará un premio

Reglas:
- Pueden usar cualquier función/clase de Python
- Pueden usar todos los datos proporcionados (incluyendo "genres.csv")
- No pueden usar datos adicionales a los dados
- Las predicciones deben estar en el Google Drive a la hora especificada
'''


In [0]:
# Codigo para hacer csv para evaluar performance
from google.colab import files

# Este código asume que tus predicciones están en un pd.Dataframe llamdo predictions
predictions.to_csv('heldout_pred.csv', index=False)

files.download('heldout_pred.csv')  # from colab to browser download